In [15]:
import pandas as pd # Data analysis and manipultion tool
import numpy as np # Fundamental package for linear algebra and multidimensional arrays
import tensorflow as tf # Deep Learning Tool
import os # OS module in Python provides a way of using operating system dependent functionality
import cv2 # Library for image processing
from sklearn.model_selection import train_test_split # For splitting the data into train and validation set
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
import seaborn as sns

In [16]:
from imutils import build_montages
from imutils import paths
import random
import cv2

In [17]:
imagePaths = list(paths.list_images("G:\PYTHON\Final year project\Final\covid_image_data\covid_image_data\train"))
random.shuffle(imagePaths)
imagePaths = imagePaths[:21]

In [18]:
# initialize the list of images
images = []
# loop over the list of image paths
for imagePath in imagePaths:
    image = cv2.imread(imagePath)
    images.append(image)
# load the image and update the list of images
# construct the montages for the images
montages = build_montages(images, (128, 196), (7, 3))

In [19]:
labels = pd.read_csv("G:\PYTHON\Final year project\Final\covid_image_data\covid_image_data\Training_set_covid.csv")

labels.head()
file_paths = [[fname, 'G:/PYTHON/Final year project/Final/covid_image_data/covid_image_data/train/' + fname] for fname in labels['filename']]

if len(labels) == len(file_paths):
    print('Number of labels i.e. ', len(labels), 'matches the number of filenames i.e. ', len(file_paths))
else:
    print('Number of labels does not match the number of filenames')

images = pd.DataFrame(file_paths, columns=['filename', 'filepaths'])
images.head()


train_data = pd.merge(images, labels, how = 'inner', on = 'filename')
train_data.head()

Number of labels i.e.  3479 matches the number of filenames i.e.  3479


,filename,filepaths,label
0,Image_1.jpg,G:/PYTHON/Final year project/Final/covid_image...,1
1,Image_2.jpg,G:/PYTHON/Final year project/Final/covid_image...,0
2,Image_3.jpg,G:/PYTHON/Final year project/Final/covid_image...,0
3,Image_4.jpg,G:/PYTHON/Final year project/Final/covid_image...,0
4,Image_5.jpg,G:/PYTHON/Final year project/Final/covid_image...,0


In [20]:
test_image_order = pd.read_csv("G:\PYTHON\Final year project\Final\covid_image_data\covid_image_data\Testing_set_covid.csv")
test_image_order.head()

file_paths_test = [[fname, 'G:/PYTHON/Final year project/Final/covid_image_data/covid_image_data/test/' + fname] for fname in test_image_order['filename']]

test_images = pd.DataFrame(file_paths_test, columns=['filename', 'filepaths'])
test_images.head()

,filename,filepaths
0,Image_1.jpg,G:/PYTHON/Final year project/Final/covid_image...
1,Image_2.jpg,G:/PYTHON/Final year project/Final/covid_image...
2,Image_3.jpg,G:/PYTHON/Final year project/Final/covid_image...
3,Image_4.jpg,G:/PYTHON/Final year project/Final/covid_image...
4,Image_5.jpg,G:/PYTHON/Final year project/Final/covid_image...


In [21]:
data = []
image_size = 224
test_pixel_data = []
import matplotlib.pyplot as plt

for i in range(len(train_data)):
    img_array = plt.imread(train_data['filepaths'][i], cv2.COLOR_BGR2RGB)

    new_img_array = cv2.resize(img_array, (image_size, image_size))
    #new_img_array = cv2.GaussianBlur(new_img_array,(5,5),0)
    data.append([new_img_array, train_data['label'][i]])


In [22]:
for i in range(len(test_images)):
    img_array = cv2.imread(test_images['filepaths'][i], cv2.COLOR_BGR2RGB)
    if img_array is None:
        print('Wrong path:', test_images['filepaths'][i])
    new_img_array = cv2.resize(img_array, (image_size, image_size))
    #new_img_array = cv2.GaussianBlur(new_img_array,(5,5),0)
    test_pixel_data.append(new_img_array)

In [23]:
np.random.shuffle(data)

In [25]:
x = []
y = []

counter = 406

for image in data:
    x.append(image[0])
    y.append(image[1])

# converting x & y to numpy array as they are list
x = np.array(x)
y = np.array(y)

In [26]:
np.unique(y, return_counts=True)

(array([0, 1], dtype=int64), array([3073,  406], dtype=int64))

In [27]:
pd.Series(y).value_counts()

0    3073
1     406
dtype: int64

In [28]:
X_train, X_val, y_train, y_val = train_test_split(x,y,test_size=0.2, random_state = 42,stratify=y)

In [29]:
test_pixel_data = np.array(test_pixel_data)

test_pixel_data = test_pixel_data/255

print(X_train.shape,X_val.shape,test_pixel_data.shape)

(2783, 224, 224, 3) (696, 224, 224, 3) (870, 224, 224, 3)


In [38]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0/255.0)

val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0/255.0)

train_generator = train_datagen.flow(X_train,y_train,batch_size=8)

val_generator = val_datagen.flow(X_val,y_val,batch_size=8)

In [39]:
pd.Series(y_val).value_counts(normalize=True)

0    0.883621
1    0.116379
dtype: float64

In [40]:
print(X_train.shape,X_val.shape)

(2783, 224, 224, 3) (696, 224, 224, 3)


In [5]:
import keras.backend as K

def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [18]:
cbs = [tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=1, min_lr=1e-5, verbose=0),
tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=1e-6, patience=10, verbose=1, mode='auto')]

In [19]:
from tensorflow.keras.applications import VGG16,VGG19,NASNetMobile,DenseNet201
from keras.layers import *
from keras.models import Sequential
from keras.regularizers import *
from tensorflow.keras.optimizers import Adam

In [20]:
def build_model():
    model = Sequential()

    conv_base = VGG16(input_shape=(224,224,3), include_top=False, pooling='max',weights='imagenet')
    model.add(conv_base)
    model.add(BatchNormalization())
    model.add(Dense(2048, activation='relu', kernel_regularizer=l1_l2(0.01)))
    model.add(BatchNormalization())
  # model.add(Dense(2048, activation='relu', kernel_regularizer=l1_l2(0.01)))
  # model.add(BatchNormalization())
  # model.add(Dense(2048, activation='relu', kernel_regularizer=l1_l2(0.01)))
  # model.add(BatchNormalization())
  # model.add(Dense(1024, activation='relu', kernel_regularizer=l1_l2(0.01)))
  # model.add(BatchNormalization())
    model.add(Dense(1, activation='sigmoid'))
    for layer in model.layers:
        layer.trainable = True
 
    model.compile(loss='binary_crossentropy', optimizer=Adam(lr=1e-4), metrics=['accuracy',"Precision","Recall"])

    return model

In [21]:
model = build_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 512)               14714688  
_________________________________________________________________
batch_normalization (BatchNo (None, 512)               2048      
_________________________________________________________________
dense (Dense)                (None, 2048)              1050624   
_________________________________________________________________
batch_normalization_1 (Batch (None, 2048)              8192      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 2049      
Total params: 15,777,601
Trainable params: 15,772,481
Non-trainable params: 5,120
_________________________________________________________________


C:\Users\Charan Chandra\anaconda3\envs\gputest\lib\site-packages\keras\optimizer_v2\optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [23]:
with tf.device("GPU"):
    model.fit(train_generator,steps_per_epoch=87,epochs=100,validation_data=val_generator,validation_steps=22,callbacks=[cbs],class_weight={0:1,1:2})

Epoch 1/100
87/87 [==============================] - 38s 434ms/step - loss: 218.6167 - accuracy: 0.6825 - precision: 0.2464 - recall: 0.8395 - val_loss: 186.8132 - val_accuracy: 0.1136 - val_precision: 0.1136 - val_recall: 1.0000
Epoch 2/100
87/87 [==============================] - 37s 426ms/step - loss: 156.6008 - accuracy: 0.8233 - precision: 0.4101 - recall: 0.8022 - val_loss: 130.6666 - val_accuracy: 0.1136 - val_precision: 0.1136 - val_recall: 1.0000
Epoch 3/100
87/87 [==============================] - 34s 386ms/step - loss: 106.3279 - accuracy: 0.8391 - precision: 0.3810 - recall: 0.7273 - val_loss: 85.3420 - val_accuracy: 0.1307 - val_precision: 0.1198 - val_recall: 0.7692
Epoch 4/100
87/87 [==============================] - 34s 387ms/step - loss: 67.3850 - accuracy: 0.8734 - precision: 0.4435 - recall: 0.7432 - val_loss: 51.6617 - val_accuracy: 0.8977 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 5/100
87/87 [==============================] - 33s 377ms/step - loss:

Epoch 37/100
87/87 [==============================] - 37s 425ms/step - loss: 1.9439 - accuracy: 0.9282 - precision: 0.7019 - recall: 0.7935 - val_loss: 2.3801 - val_accuracy: 0.6932 - val_precision: 0.3117 - val_recall: 0.9600
Epoch 38/100
87/87 [==============================] - 37s 424ms/step - loss: 1.8978 - accuracy: 0.9237 - precision: 0.6024 - recall: 0.7143 - val_loss: 3.0072 - val_accuracy: 0.1364 - val_precision: 0.1111 - val_recall: 1.0000
Epoch 39/100
87/87 [==============================] - 37s 425ms/step - loss: 1.8366 - accuracy: 0.9339 - precision: 0.6857 - recall: 0.8471 - val_loss: 1.7815 - val_accuracy: 0.9432 - val_precision: 0.7619 - val_recall: 0.7619
Epoch 40/100
87/87 [==============================] - 37s 425ms/step - loss: 1.8176 - accuracy: 0.9325 - precision: 0.6882 - recall: 0.7805 - val_loss: 3.2326 - val_accuracy: 0.2670 - val_precision: 0.1513 - val_recall: 1.0000
Epoch 41/100
87/87 [==============================] - 37s 425ms/step - loss: 1.7695 - accura

In [136]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'


'1'

In [24]:
model.save("G:\\PYTHON\\Final year project\\Final\\covid_image_data\\covid_image_data\\vgg_16_1.h5")

In [128]:
preds = model.predict(X_val/255.0)
predictions = []
for i in preds:
    if i[0] > 0.5:
        predictions.append(1)
    else:
        predictions.append(0)

In [26]:
from sklearn.metrics import f1_score,confusion_matrix
print(f1_score(y_val,predictions))
print(confusion_matrix(y_val,predictions))

0.5344827586206897
[[611   4]
 [ 50  31]]


In [30]:
preds = model.predict(test_pixel_data)

In [126]:
predictions = []
for i in preds:
    if i[0] > 0.8:
        predictions.append(1)
    else:
        predictions.append(0)

In [51]:
pd.Series(predictions).value_counts()

0    837
1     33
dtype: int64

In [127]:
from sklearn.metrics import f1_score,confusion_matrix
print(f1_score(y_val,predictions))
print(confusion_matrix(y_val,predictions))

ValueError: Found input variables with inconsistent numbers of samples: [696, 870]

In [53]:
result = pd.DataFrame(predictions,columns=['prediction'])

result.value_counts()

prediction
0             837
1              33
dtype: int64

In [2]:
from keras.models import load_model

In [3]:
model = load_model("G:\\PYTHON\\Final year project\\Final\\covid_image_data\\covid_image_data\\vgg_16_1.h5",
                  )

In [12]:
# get sample image to test.
import numpy as np
import cv2

img_covid = 'G:\\PYTHON\\Final year project\\Dataset\\COVID-19_Radiography_Dataset\\COVID\\COVID-567.png'
img_normal = 'G:\\PYTHON\\Final year project\\Dataset\\COVID-19_Radiography_Dataset\\Normal\\Normal-50.png'

img_array = cv2.imread(img_normal)
new_img_array = cv2.resize(img_array, (224, 224))
dt = []
dt.append(new_img_array)
X = np.array(dt)
X = X/255
val = model.predict(X)
#print("Confidence : ",round(val[0][0]*100,2),"%")
if val > 0.5:
    print("COVID")
else:
    print("NORMAL")

NORMAL
